In [1]:
import mnist
import numpy as np

In [2]:
xtrain = mnist.train_images()[:100]
ytrain = mnist.train_labels()[:100]
xtest = mnist.test_images()[:100]
ytest = mnist.test_labels()[:100]

In [3]:
class Myconv2d():
    def __init__(self, num_filters):
        ''' Assumes 3*3 kernel size ''' 
        self.num_filters = num_filters
        self.filters = np.random.randn(num_filters, 3, 3)/9

    def forward(self, input):
        ''' input is 2d array '''
        xh, xw = input.shape
        fh, fw = 3, 3 
        yh, yw = xh - fh + 1, xw - fw + 1
        output = np.zeros((yh, yw, self.num_filters))

        for row in range(yh):
            for column in range(yw):
                output[row, column] = np.sum(input[row : row + fh, column : column + fw] * self.filters, axis = (1, 2))
        
        return output

In [12]:
class Mymaxpool2d():
    def __init__(self, filter_size):
        self.filter_size = filter_size
        
    def forward(self, input):
        self.last_input = input

        ih, iw, num_filters = input.shape
        fh, fw = self.filter_size, self.filter_size
        yh, yw = int(ih/self.filter_size), int(iw/self.filter_size) 
        output = np.zeros(shape = (yh, yw, num_filters))
        for row in range(yh):
            for column in range(yw):
                output[row, column] = np.amax(image[row*fh : row*fh + fh, column*fw : column*fw + fw], axis=(0, 1))
        return output

In [16]:
class Mysoftmax():
    def __init__(self, input_len, nodes):
        self.input_len, self.nodes = input_len, nodes
        self.weights = np.random.randn(self.input_len, self.nodes)/self.input_len
        self.bias = np.zeros(self.nodes)

    def forward(self, input):
        self.last_input = input
        self.last_input_shape = input.shape
        
        input = input.flatten()
        pre_activation = np.dot(input, self.weights) + self.bias
        
        self.last_pre_activation = pre_activation
        activation = np.exp(pre_activation)
        return activation/np.sum(activation, axis = 0)

In [19]:
## Checking with sample image
image = xtrain[0]/255 - 0.5
conv = Myconv2d(8)
out = conv.forward(image)
print(f'conv output shape : {out.shape}')
pool = Mymaxpool2d(2)
out = pool.forward(out)
print(f'maxpool output shape : {out.shape}')
softmax_input_shape = 13*13*8
softmax = Mysoftmax(softmax_input_shape, 10)
out = softmax.forward(out)
print(f'softmax output shape : {out.shape}')

conv output shape : (26, 26, 8)
maxpool output shape : (13, 13, 8)
softmax output shape : (10,)


In [26]:
# foward_only
conv = Myconv2d(8)
pool = Mymaxpool2d(2)
softmax_input_shape = 13*13*8
softmax = Mysoftmax(softmax_input_shape, 10)

for epoch in range(1):
    acc_epoch = 0
    for image, label in zip(xtrain, ytrain):
        image = image/255 - 0.5
        
        # Forward Prop
        out = conv.forward(image)
        out = pool.forward(out)
        out = softmax.forward(out)
        
        # CE Loss, Accuracy
        loss = -np.log(out[label])
        acc = 1 if np.argmax(out) == label else 0
        acc_epoch += acc
    print(f'epoch : {epoch} loss : {loss} accuracy {acc_epoch/len(ytrain)}')

epoch : 0 loss : 2.3164589512799334 accuracy 0.08
